In [1]:
import spacy
import json
from spacy import displacy
import re
import pandas as pd
from configparser import ConfigParser
import string

/data/mmortgage/NERAPI/NER/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# def get_annotation(file):
    
#     filename,ents= file
#     #print(filename)
#     data = json.load(open('./OCR_Files/'+filename, encoding="utf-8"))
#     txt = ''
#     annotations=[]
#     for pages in data['analyzeResult']['readResults']:
#         txt=''
#         for line in pages['lines']:
#             txt = txt+' '+line['text']
#         #print(txt,'\n\n')
#         para = txt
#         txt = txt.lower()
#         ner_tag = {}
#         tags=[]
#         for ent in ents:
#             txt2find,tag = ent
#             txt2find = txt2find.lower()
#             start=0
#             for i,character in enumerate(txt):
#                 flag=0
#                 temp=''
#                 if character == txt2find[0]:
#                     start = i
#                     j=i
#                     k=0
#                     while((k<len(txt2find))&(j<len(txt)-1)):
#                         if(txt[j]==txt2find[k]):
#                             temp=temp+txt[j]
#                             j=j+1
#                             k=k+1
#                         elif(txt2find[k] in string.punctuation):
#                             k=k+1
#                         elif(txt[j] in string.punctuation):
#                             temp=temp+txt[j]
#                             j=j+1
#                         else:
#                             end=j
#                             break
#                 if ((len(temp)>=len(txt2find))):
#                     tags.append((start, start+len(temp),tag))
#         if len(tags)!=0:
#             ner_tag['entities'] = tags
#             annotations.append((para,ner_tag))
#     return(annotations)

In [3]:
# from ast import literal_eval
# annotations = []
# with open('./train_both.txt', 'r', encoding="utf-8") as f:
#     data = f.read().split(',\n')
# for d in data:
#     #print(literal_eval(d))
#     annotations.append(literal_eval(d))
# len(annotations)

In [4]:
# annotations[340]

In [5]:
# SAMPLES = []
# length = []
# count = 0
# i=0
# for annot in annotations:
#     #print(i)
#     i=i+1
#     entities = []
#     for entries in annot[1]:
#         entities.append(entries[1])
#     entities = list(set(entities))
    
#     try:
#         annotation = get_annotation(annot)
#         if len(annotation)!=0:
#             SAMPLES = SAMPLES + annotation
#         else:
#             count=count+1
#             #print(annot[0])
#     except FileNotFoundError:
#         pass
# print('Total samples ->', len(SAMPLES))
# print('Total no match found ->', count)

In [97]:
df = pd.read_csv('./ner.csv')
df['NER'].head()

0    ('Regions Bank Sandlake & Bay Hill 8061 Via De...
1    ('USAA® USAA Federal Savings Bank 10750 McDerm...
2    ('Aug 19 2022 02:56PM HP Fax page 2 CENTURY CR...
3    ('Page 1 of 3 NAVY O FEDERAL Credit Union PO B...
4    ('First Security Bank FSB LENGER MEMBER FDIC C...
Name: NER, dtype: object

In [106]:
import ast

SAMPLES = []
count=0
for value in df['NER'].values:
    if len(ast.literal_eval(value)[1]['entities'])!=0:
        SAMPLES.append(ast.literal_eval(value))

In [107]:
len(SAMPLES)

131

### Spacy input format

In [108]:
SAMPLES[2]

('Aug 19 2022 02:56PM HP Fax page 2 CENTURY CREDIT UNION centurycu.org Member Statement Account Number: 0318579000 Statement Period: 09/01/2020 to 12/31/2020 1540 Lemay Ferry Road Saint Louis, MO 63125 Phone: 1-400-880-0640 | www.camrycu.org Page 1 of 2 RETURN SERVICE REQUESTED CHAD R BRADLEY 409 WEST 6TH ST PORTAGEVILLE MO 63873 ACCOUNT SUMMARY Shares Starting Balance Total Deposits Total Withdrawals 0001 PRIMARY SAVINGS 25.00 0.00 0.00 0080 ESCROW SAVINGS 818.03 628.29 -892.23 Loans Total Payments Starting Balance Total Advances 0001 USED AUTO 9,905.01 1,080.00 0.00 0050 30 YEAR FIRST MORTGAGE ESCROW 127,630.56 2,700.00 0.00 ID 0001 - PRIMARY SAVINGS Dividends Paid YTD: 0.00 Date Transaction Description Withdrawal Deposit Balance 09/01/2020 Balance Forward 25.0 12/31/2020 Ending Balance 25.00 ID 0080 - ESCROW SAVINGS Dividends Paid YTD: 0.00 Date Transaction Description Amount Principal Charge Fee Balance Balance Forward 10/01/2020 9,905.01 10/27/2020 Payments by Check -360.00 -319.0

### TRAIN TEST SPLIT ***

In [109]:
import random
random.shuffle(SAMPLES)

In [110]:
TRAIN_DATA = SAMPLES[:80]
VAL_DATA = SAMPLES[81:101]
TEST_DATA = SAMPLES[101:136]

In [111]:
TEST_DATA[1]

('USAA® USAA Federal Savings Bank 10750 McDermott Freeway San Antonio, TX 78288-0544 USAA CLASSIC CHECKING for Account Number: 0158116623 Statement Period: 07/29/2022 to 08/30/2022 PATRICK J BURK SUZANNE P BURK 444 DEVERAUX DR WILMINGTON NC 28412 Activity Summary $6,832.68 $11,429.19 $15,573.55 Beginning Balance 7 Deposits/Credits Service Charges and ATM Service Fee Ending Balance $2,688.32 Total For This Period Total Year-to-Date Fee Total Overdraft (OD) Fees N/A N/A Total Non-Sufficient Funds (NSF) Fees $0.00 $0.00 Note: Fee reversals/refunds will not be reflected in this table. They will be listed in the transaction section below. Note: Transactions occurring on the first day and last day of the statement cycle are included in this statement. Transactions Description Debits Credits Date Balance 07/29 Beginning Balance $6,832.68 07/29 ACH DEP 080122 $152.64 $6,985.32 VACP TREAS 310 XXVA BENEF .... **** 3600 $6,978.69 07/29 POS DEBIT 072922 5411072922 $6.63 FOOD LION #1404 3905 I WILM

In [112]:
print('no. of train samples: ', len(TRAIN_DATA))
print('no. of val samples: ', len(VAL_DATA))
print('no. of test samples: ', len(TEST_DATA))

no. of train samples:  80
no. of val samples:  20
no. of test samples:  30


In [113]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

#nlp = spacy.blank("en") # load a new spacy model
#nlp = spacy.load("en_core_web_sm") # load other spacy model
nlp = spacy.load('en_core_web_trf', exclude=['tagger', 'parser', 'attribute_ruler', 'lemmatizer'])
#nlp = spacy.load('en_core_web_trf')

def spacy_format(DATA, val_data):
    db = DocBin() # create a DocBin object

    for text, annot in tqdm(DATA): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            #print(span)
            if span is None:
                #print("Skipping entity")
                pass
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
    
    if val_data==False:
        db.to_disk("./train.spacy") # save the docbin object
    else:
        db.to_disk("./val.spacy")

In [114]:
spacy_format(TRAIN_DATA, val_data=False)
spacy_format(VAL_DATA, val_data=True)

100%|██████████| 20/20 [00:00<00:00, 285.27it/s]


In [34]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

/usr/bin/python: No module named spacy


In [ ]:
!python -m spacy train config.cfg --output /mnt/spacy/ --paths.train ./train.spacy --paths.dev ./val.spacy --gpu-id 0

In [ ]:
ls

### EVALUATION

In [71]:
import spacy
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training.example import Example

def evaluate(ner_model, samples):
    scorer = Scorer(ner_model)
    examples = []
    for input_, annot in samples:
        pred = ner_model(input_)
        example = Example.from_dict(pred, annot)
        examples.append(example)
    scores = scorer.score(examples)    
    return scores

In [72]:
import numpy as np
ner = spacy.load('/mnt/spacy/model-best')

In [73]:
evaluate(ner, TEST_DATA)

/data/mmortgage/NERAPI/NER/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Valley TAKE THE NEXT STEPS IN BECOMING A HOMEOWNER..." with entities "[(0, 6, 'ORG'), (224, 230, 'ORG'), (311, 317, 'ORG...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/data/mmortgage/NERAPI/NER/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "USAA® USAA Federal Savings Bank 10750 McDermott Fr..." with entities "[(6, 31, 'ORG'), (199, 213, 'PERSON'), (1008, 1022...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/data/mmortgage/NERAPI/NER/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: Us

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'ents_p': 0.7982456140350878,
 'ents_r': 0.6453900709219859,
 'ents_f': 0.7137254901960784,
 'ents_per_type': {'PERSON': {'p': 0.7678571428571429,
   'r': 0.8431372549019608,
   'f': 0.8037383177570093},
  'ADDRESS': {'p': 0.7083333333333334,
   'r': 0.7391304347826086,
   'f': 0.723404255319149},
  'ORG': {'p': 0.9117647058823529,
   'r': 0.4626865671641791,
   'f': 0.6138613861386137}}}

In [92]:
TEST_DATA[13]

('From:THE UPS STORE 9075632276 08/31/2022 12:31 #384 P.007/015 COPY STATEMENT OF ACCOUNT 4786073 ACCOUNT STATEMENT PERIOD FROM 07-15-22 THROUGH 08-14-22 PAGE 1 ALASKA USA FEDERAL CREDIT UNION PO Box 196613 Anchorage, Alaska 99519-6613 www.alaskausa.org EARNEST R CARTER JR PO BOX 212256 ANCHORAGE AK 99521-2256 - ACCOUNT SUMMARY DIVIDENDS WITHHOLDING SHARE ACCOUNTS PREVIOUS NEW YEAR-TO-DATE YEAR-TO-DATE BALANCE BALANCE 10 SHARE SAV 896.76 711.64 0.36 0.00 70 251.96 0.00 0.00 199.93 CONV CHEK TOTAL SHARE ACCOUNTS 911.57 EXPANDED TRANSACTION DESCRIPTION AMOUNT BALANCE PREVIOUS BALANCE 896.76 PAI ISO 239 W 4TH ST ANCHORAGE AK 83.00 - 813.76 5.01 - 808.75 WITHDRAWAL TRANSFER TO SHARE 70 112.83 - 695.92 WITHDRAWAL ULTRABRANCH-PC TRANSFER 200.00 - 495.92 TO SHARE 70 GREAT NORTHE-K13 2341 CINNABAR 392.92 103.00 - LOOP US 07-21 07-21 WITHDRAWAL AT ATM #005904 TO SHARE 70 53.00 - 2086.20 1100.00 - 339.92 2426.12 1326.12 07-26 07-27 07-27 07-26 WITHDRAWAL AT ATM #000000001787 07-26 DEPOSIT ACH SS

In [93]:
doc = ner(TEST_DATA[13][0])

for ent in doc.ents:
    if ent.label_ == 'ORG':
        print("ORG: ",ent.text)
    if ent.label_ == 'PERSON':
        print("PERSON: ",ent.text)
    if ent.label_ == 'ADDRESS':
        print("ADDRESS: ",ent.text)

ORG:  ALASKA USA FEDERAL CREDIT UNION
PERSON:  EARNEST R CARTER JR


In [96]:
txt = TEST_DATA[13][0]
txt[272:309]

'PO BOX 212256 ANCHORAGE AK 99521-2256'